In [1]:
%cd ~/fast/ProtoTSNetDPL/

/home/jovyan/fast/ProtoTSNetDPL


/home/jovyan/.conda/envs/dpl2/lib/python3.11/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [2]:
from scipy import signal
import numpy as np
import torch
import json

from deepproblog.dataset import Dataset as DPLDataset, DataLoader
from deepproblog.query import Query
from deepproblog.network import Network
from deepproblog.model import Model
from deepproblog.engines import ExactEngine
from deepproblog.train import train_model
from deepproblog.evaluate import get_confusion_matrix, get_fact_accuracy
from problog.logic import Term, Constant, list2term, Not

from model import ProtoTSNet
from autoencoder import RegularConvEncoder
from artificial_datasets_DPL import ArtificialProtosDataset, ArtificialProtosDatasetRandomShift, Queries

from matplotlib import pyplot as plt

from pathlib import Path

from datasets_utils import ds_load

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
%env CUDA_VISIBLE_DEVICES=0
torch.cuda.set_per_process_memory_fraction(fraction=0.5, device=0)

env: CUDA_VISIBLE_DEVICES=0


In [64]:
dataset = ArtificialProtosDatasetRandomShift(50, num_feat=2, classes=2, feature_noise_power=0)

In [ ]:
for i in range(10):
    idx = i
    print(f'Label: {dataset.get_label(idx)}')
    plt.plot(dataset[[idx]].T)
    plt.show()

In [87]:
test_dataset = ArtificialProtosDataset(50)
test_queries = Queries(test_dataset, "test")

In [4]:
libras = ds_load(Path('./datasets'), 'Libras')
test_queries = Queries(libras.test, "test")

Detected number of classes in Queries: 15


In [5]:
protos_per_class = 1
latent_features = 32
num_features = 2
ts_len = 45
proto_len_latent = 5
# num_features = 3
num_classes = 15
NOISE_POWER = 0.1

print('Preparing ProtoTSNet...')
autoencoder = RegularConvEncoder(num_features=num_features, latent_features=latent_features, padding='same')
encoder = autoencoder.encoder
net = ProtoTSNet(
    cnn_base=encoder,
    for_deepproblog=True,
    num_features=num_features,
    ts_sample_len=ts_len,
    proto_num=protos_per_class*num_classes,
    latent_features=latent_features,
    proto_len_latent=proto_len_latent,
    num_classes=num_classes,
)

dpl_net = Network(net, "ptsnet", batching=True)
# dpl_net.cuda()

print('Loading logic file...')
model = Model("proto_logic.pl", [dpl_net])
model.set_engine(ExactEngine(model))

Preparing ProtoTSNet...
Loading logic file...


In [6]:
model.load_state('./experiments/LibrasTrainableLogic/models/last-epoch.pth')
model.eval()
model.add_tensor_source("test", libras.test)

In [20]:
print(test_queries.to_query(0)[0])
print(Query(Term('excl_is_class', Term('ts0'), Term('c0')), {Term('ts0'): Term('tensor', Term('test', Constant(0)))}))

(1.0::excl_is_class(ts0,c0), {ts0: tensor(test(0))})
(1.0::excl_is_class(ts0,c0), {ts0: tensor(test(0))})


In [8]:
libras.test.get_label(50)

np.True_

In [14]:
ts = 50
c = 'c10'
# print(model.solve([Query(Term('excl_is_class', Term(f'ts{ts}'), Term(c)), {Term(f'ts{ts}'): Term('tensor', Term('test', Constant(ts)))})]))
for c in range(15):
    print(model.solve([Query(Term('excl_is_class', Term(f'ts{ts}'), Term(f'c{c}')), {Term(f'ts{ts}'): Term('tensor', Term('test', Constant(ts)))}, p=float(libras.test.get_label(ts) == c))]))
print()
# print(model.solve([Query(Term('is_class', Term(f'ts{ts}'), Term(c)), {Term(f'ts{ts}'): Term('tensor', Term('test', Constant(ts)))})]))
for c in range(15):
    print(model.solve([Query(Term('is_class', Term(f'ts{ts}'), Term(f'c{c}')), {Term(f'ts{ts}'): Term('tensor', Term('test', Constant(ts)))})]))
print()
for c in range(15):
    print(model.solve([Query(Term('is_class', Term(f'ts{ts}'), Term(f'c{c}')), {Term(f'ts{ts}'): Term('tensor', Term('test', Constant(ts)))})]))
# print(model.solve([Query(Not('not', Term('is_class', Term(f'ts{ts}'), Term('c0')), {Term(f'ts{ts}'): Term('tensor', Term('test', Constant(ts)))}))]))
print(model.solve([Query(Term('has_proto', Term(f'ts{ts}'), Term('p10')), {Term(f'ts{ts}'): Term('tensor', Term('test', Constant(ts)))})]))
print(model.solve([Query(Term('connected', Term(f'p10'), Term(c)))]))
print(model.solve([Query(Not('not', Term('connected', Term(f'p10'), Term(c))))]))

[{excl_is_class(tensor(test(50)),c0): tensor(0.6397, grad_fn=<AddBackward0>)}]
[{excl_is_class(tensor(test(50)),c1): tensor(0.6454, grad_fn=<AddBackward0>)}]
[{excl_is_class(tensor(test(50)),c2): tensor(0.6457, grad_fn=<AddBackward0>)}]
[{excl_is_class(tensor(test(50)),c3): tensor(0.6458, grad_fn=<AddBackward0>)}]
[{excl_is_class(tensor(test(50)),c4): tensor(0.6458, grad_fn=<AddBackward0>)}]
[{excl_is_class(tensor(test(50)),c5): tensor(0.6458, grad_fn=<AddBackward0>)}]
[{excl_is_class(tensor(test(50)),c6): tensor(0.6457, grad_fn=<AddBackward0>)}]
[{excl_is_class(tensor(test(50)),c7): tensor(0.6457, grad_fn=<AddBackward0>)}]
[{excl_is_class(tensor(test(50)),c8): tensor(0.6458, grad_fn=<AddBackward0>)}]
[{excl_is_class(tensor(test(50)),c9): tensor(0.6457, grad_fn=<AddBackward0>)}]
[{excl_is_class(tensor(test(50)),c10): tensor(0.6458, grad_fn=<AddBackward0>)}]
[{excl_is_class(tensor(test(50)),c11): tensor(0.6457, grad_fn=<AddBackward0>)}]
[{excl_is_class(tensor(test(50)),c12): tensor(0.64

In [74]:
next(net.parameters()).shape

torch.Size([2, 32, 20])

In [75]:
# idx = 7
# print(f'Label: {test_dataset.get_label(idx)}')
# plt.plot(test_dataset[[idx]][0])

In [79]:
idx = 3
print(f'Label: {test_dataset.get_label(idx)}')
print(model.solve([test_queries.to_query(2*idx+0)]))
print(model.solve([test_queries.to_query(2*idx+1)]))

Label: 1
[{is_class(tensor(test(3)),c0): tensor(0.0596, grad_fn=<SelectBackward0>)}]
[{is_class(tensor(test(3)),c1): tensor(0.9075, grad_fn=<SelectBackward0>)}]
